# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [1]:
import pandas as pd
import os
import plotly.express as px


# Notebook Presentation

In [2]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
# Prefer local file if available to avoid network issues
if os.path.exists('apps.csv'):
    df_apps = pd.read_csv('apps.csv')
else:
    # Fallback to URL if local file is missing
    url = 'https://raw.githubusercontent.com/rudil24/pythonCourse/main/solutions/day076/apps.csv'
    df_apps = pd.read_csv(url)

# Read the Dataset

# Data Cleaning

**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [4]:
df_apps.shape # get the shape of the dataframe in (rows, cols)

(10841, 12)

In [5]:
df_apps.columns # what are the column names of the dataframe?

Index(['App', 'Category', 'Rating', 'Reviews', 'Size_MBs', 'Installs', 'Type',
       'Price', 'Content_Rating', 'Genres', 'Last_Updated', 'Android_Ver'],
      dtype='str')

In [6]:
df_apps.sample(5)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
221,CZ-70 (CZ-50) pistol explained,BOOKS_AND_REFERENCE,NaN,2,17.00,10,Paid,$5.99,Everyone,Books & Reference,"November 2, 2016",1.6 and up
8143,Super Jabber Jump 3,GAME,4.30,34494,23.00,"5,000,000",Free,0,Everyone 10+,Arcade,"July 11, 2018",2.3 and up
1004,HD Themes Toyota Cruiser70,PERSONALIZATION,NaN,0,15.00,100,Free,0,Teen,Personalization,"October 3, 2016",2.3.3 and up
9068,Zoosk Dating App: Meet Singles,DATING,4.00,516801,11.00,"10,000,000",Free,0,Mature 17+,Dating,"August 2, 2018",Varies with device
9607,"Jaumo Dating, Flirt & Live Video",SOCIAL,4.40,900064,13.00,"10,000,000",Free,0,Mature 17+,Social,"July 31, 2018",4.1 and up


### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns.

In [7]:
df_apps.drop(['Last_Updated', 'Android_Ver'], axis=1, inplace=True)

### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows.

In [8]:
print(df_apps['Rating'].isnull().sum())

1474


In [9]:
df_apps_clean = df_apps.dropna(subset=['Rating']).copy()
print(df_apps_clean.shape)

(9367, 10)


### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`.


In [10]:
print(df_apps_clean.duplicated().sum())

476


In [11]:
print(df_apps_clean[df_apps_clean['App'] == 'Instagram'].shape[0])

4


In [12]:
df_apps_clean.drop_duplicates(inplace=True)

In [13]:
print(df_apps_clean.shape)

(8891, 10)


# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [14]:
df_apps_clean.sort_values(by='Rating', ascending=False).head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
21,KBA-EZ Health Guide,MEDICAL,5.00,4,25.00,1,Free,0,Everyone,Medical
1573,FHR 5-Tier 2.0,MEDICAL,5.00,2,1.20,500,Paid,$2.99,Everyone,Medical
1096,BG Guide,TRAVEL_AND_LOCAL,5.00,3,2.40,100,Free,0,Everyone,Travel & Local
1095,Morse Player,FAMILY,5.00,12,2.40,100,Paid,$1.99,Everyone,Education
1092,DG TV,NEWS_AND_MAGAZINES,5.00,3,5.70,100,Free,0,Everyone,News & Magazines


<p style="color:#d4e065">possible problem: low amounts of reviews can skew to a high or low rating. we should probably combine rating with reviews with installs (or throw out low-reviewed apps)</p>

# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please?
<p style="color:#d4e065">Looks like it gets capped at 100MB!</p>

In [15]:
# Find the 5 largest apps by size
df_apps_clean.nlargest(5, 'Size_MBs')[['App', 'Size_MBs', 'Category']]

,App,Size_MBs,Category
1795,Navi Radiography Pro,100.00,MEDICAL
3144,Vi Trainer,100.00,HEALTH_AND_FITNESS
4176,Car Crash III Beam DH Real Damage Simulator 2018,100.00,GAME
7926,Post Bank,100.00,FINANCE
7927,The Walking Dead: Our World,100.00,GAME


# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?

In [16]:
df_apps_clean.sort_values(by=['Reviews'], ascending=[False]).head(10)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10805,Facebook,SOCIAL,4.10,78158306,5.30,"1,000,000,000",Free,0,Teen,Social
10811,Facebook,SOCIAL,4.10,78128208,5.30,"1,000,000,000",Free,0,Teen,Social
10785,WhatsApp Messenger,COMMUNICATION,4.40,69119316,3.50,"1,000,000,000",Free,0,Everyone,Communication
10797,WhatsApp Messenger,COMMUNICATION,4.40,69109672,3.50,"1,000,000,000",Free,0,Everyone,Communication
10808,Instagram,SOCIAL,4.50,66577446,5.30,"1,000,000,000",Free,0,Teen,Social
10806,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10810,Instagram,SOCIAL,4.50,66509917,5.30,"1,000,000,000",Free,0,Teen,Social
10790,Messenger – Text and Video Chat for Free,COMMUNICATION,4.00,56646578,3.50,"1,000,000,000",Free,0,Everyone,Communication
10784,Messenger – Text and Video Chat for Free,COMMUNICATION,4.00,56642847,3.50,"1,000,000,000",Free,0,Everyone,Communication
10652,Clash of Clans,GAME,4.60,44893888,98.00,"100,000,000",Free,0,Everyone 10+,Strategy


In [28]:
# Get the top 100 apps by number of reviews
top_100_reviews = df_apps_clean.sort_values(by='Reviews', ascending=False).head(100)

# Filter for paid apps in the top 50 most-reviewed apps
paid_apps_top_100 = top_100_reviews[top_100_reviews['Type'] == 'Paid']
print(paid_apps_top_100)
print(f"\nNumber of paid apps in top 100: {len(paid_apps_top_100)}")

Empty DataFrame
Columns: [App, Category, Rating, Reviews, Size_MBs, Installs, Type, Price, Content_Rating, Genres]
Index: []

Number of paid apps in top 100: 0


<p style="color:#d4e065">no paid apps in the top 50 or top 100!</p>

### Apps with 5 stars that have the most reviews

In [27]:
df_apps_clean.sort_values(by=['Rating', 'Reviews'], ascending=[False, False]).head(10)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
2095,Ríos de Fe,LIFESTYLE,5.00,141,15.00,"1,000",Free,0,Everyone,Lifestyle
2438,"FD Calculator (EMI, SIP, RD & Loan Eligilibility)",FINANCE,5.00,104,2.30,"1,000",Free,0,Everyone,Finance
3115,Oración CX,LIFESTYLE,5.00,103,3.80,"5,000",Free,0,Everyone,Lifestyle
2107,Barisal University App-BU Face,FAMILY,5.00,100,10.00,"1,000",Free,0,Everyone,Education
2069,Master E.K,FAMILY,5.00,90,19.00,"1,000",Free,0,Everyone,Education
1968,CL REPL,TOOLS,5.00,47,17.00,"1,000",Free,0,Everyone,Tools
790,AJ Cam,PHOTOGRAPHY,5.00,44,2.80,100,Free,0,Everyone,Photography
1275,AI Today : Artificial Intelligence News & AI 101,NEWS_AND_MAGAZINES,5.00,43,2.30,100,Free,0,Everyone,News & Magazines
1789,Ek Vote,PRODUCTIVITY,5.00,43,6.20,500,Free,0,Everyone,Productivity
2544,CS & IT Interview Questions,FAMILY,5.00,43,3.30,"1,000",Free,0,Everyone,Education


# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings

In [29]:
ratings = df_apps_clean.Content_Rating.value_counts()
display(ratings)

Content_Rating
Everyone           7094
Teen               1022
Mature 17+          411
Everyone 10+        360
Adults only 18+       3
Unrated               1
Name: count, dtype: int64

In [30]:
fig = px.pie(labels=ratings.index, values=ratings.values)
fig.show()

In [31]:
# make the pie a bit better looking with better labels
fig = px.pie(labels=ratings.index,
values=ratings.values,
title="Content Rating",
names=ratings.index,
)
fig.update_traces(textposition='outside', textinfo='percent+label')
 
fig.show()

In [32]:
# turn a pie into a donut with the hole attribute
fig = px.pie(labels=ratings.index,
values=ratings.values,
title="Content Rating",
names=ratings.index,
hole=0.6,
)
fig.update_traces(textposition='inside', textfont_size=15, textinfo='percent')
 
fig.show()

# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install?

Check the datatype of the Installs column.

Count the number of apps at each level of installations.

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first.

In [33]:
# Check the datatype of the Installs column
print("df_apps Installs dtype:", df_apps['Installs'].dtype)
print("df_apps_clean Installs dtype:", df_apps_clean['Installs'].dtype)

# Show a sample of values to inspect formatting (commas, plus signs, etc.)
print(df_apps['Installs'].head(10).tolist())

df_apps Installs dtype: str
df_apps_clean Installs dtype: str
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']


In [34]:
# Count apps at each raw "Installs" level (as stored, e.g., "1,000,000,000")
print("Raw installs value counts:")
print(df_apps['Installs'].value_counts())

# Also produce counts after cleaning to numeric (remove commas and pluses)
installs_clean = pd.to_numeric(df_apps['Installs'].str.replace('[+,]', '', regex=True), errors='coerce')
print("\nNumeric installs value counts (sorted):")
print(installs_clean.value_counts().sort_index())

# Show how many could not be converted (if any)
print(f"\nUnconvertible install entries: {installs_clean.isna().sum()}")

Raw installs value counts:
Installs
1,000,000        1579
10,000,000       1252
100,000          1169
10,000           1054
1,000             908
5,000,000         752
100               719
500,000           539
50,000            479
5,000             477
100,000,000       409
10                386
500               330
50,000,000        289
50                205
5                  82
500,000,000        72
1                  67
1,000,000,000      58
0                  15
Name: count, dtype: int64

Numeric installs value counts (sorted):
Installs
0               15
1               67
5               82
10             386
50             205
100            719
500            330
1000           908
5000           477
10000         1054
50000          479
100000        1169
500000         539
1000000       1579
5000000        752
10000000      1252
50000000       289
100000000      409
500000000       72
1000000000      58
Name: count, dtype: int64

Unconvertible install entries: 0


In [35]:
# step 1+2: remove non-numeric chars (commas, plus) then convert to numeric (nullable int)
df_apps['Installs'] = pd.to_numeric(df_apps['Installs'].str.replace('[+,]', '', regex=True), errors='coerce').astype('Int64')
df_apps_clean['Installs'] = pd.to_numeric(df_apps_clean['Installs'].str.replace('[+,]', '', regex=True), errors='coerce').astype('Int64')

# quick verification
print("df_apps Installs dtype:", df_apps['Installs'].dtype)
print("df_apps_clean Installs dtype:", df_apps_clean['Installs'].dtype)
print("Apps with >= 1,000,000,000 installs:", (df_apps_clean['Installs'] >= 1_000_000_000).sum())
print("Apps with exactly 1 install:", (df_apps_clean['Installs'] == 1).sum())

df_apps Installs dtype: Int64
df_apps_clean Installs dtype: Int64
Apps with >= 1,000,000,000 installs: 49
Apps with exactly 1 install: 3


# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

<p style="color:#d4e065">I did the above but i put it in a new `df_price_clean` dataframe</p>

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


In [36]:
# Convert Price to numeric (without modifying original Price) and show top 20 most expensive rows
df_price_clean = df_apps_clean.copy()
df_price_clean['Price_USD'] = pd.to_numeric(df_price_clean['Price'].astype(str).str.replace('[$,]', '', regex=True), errors='coerce')

top20_price = df_price_clean.sort_values('Price_USD', ascending=False).head(20)

# Display key columns for inspection
display(top20_price[['App', 'Category', 'Type', 'Price', 'Price_USD', 'Rating', 'Reviews', 'Installs', 'Genres']])

# Quick summary
print("Unique apps in top 20:", top20_price['App'].nunique())
print("Paid entries in top 20:", (top20_price['Type'] == 'Paid').sum())
print("Entries classified as GAME in top 20:", (top20_price['Category'].str.upper() == 'GAME').sum())
print("\nPrice stats for top 20:")
print(top20_price['Price_USD'].describe())

,App,Category,Type,Price,Price_USD,Rating,Reviews,Installs,Genres
3946,I'm Rich - Trump Edition,LIFESTYLE,Paid,$400.00,400.00,3.60,275,10000,Lifestyle
5765,I am rich,LIFESTYLE,Paid,$399.99,399.99,3.80,3547,100000,Lifestyle
2775,I Am Rich Pro,FAMILY,Paid,$399.99,399.99,4.40,201,5000,Entertainment
1331,most expensive app (H),FAMILY,Paid,$399.99,399.99,4.30,6,100,Entertainment
3554,💎 I'm rich,LIFESTYLE,Paid,$399.99,399.99,3.80,718,10000,Lifestyle
3145,I am rich(premium),FINANCE,Paid,$399.99,399.99,3.50,472,5000,Finance
3221,I am Rich Plus,FAMILY,Paid,$399.99,399.99,4.00,856,10000,Entertainment
4606,I Am Rich Premium,FINANCE,Paid,$399.99,399.99,4.10,1867,50000,Finance
3114,I am Rich,FINANCE,Paid,$399.99,399.99,4.30,180,5000,Finance
2394,I am Rich!,FINANCE,Paid,$399.99,399.99,3.80,93,1000,Finance


Unique apps in top 20: 20
Paid entries in top 20: 20
Entries classified as GAME in top 20: 0

Price stats for top 20:
count    20.00
mean    304.59
std     156.00
min      29.99
25%     244.99
50%     399.99
75%     399.99
max     400.00
Name: Price_USD, dtype: float64


### The most expensive apps sub $250

In [37]:
# remove any app with Price_USD >= 250 from df_price_clean 
df_price_clean = df_price_clean[df_price_clean['Price_USD'] < 250]
top20_price = df_price_clean.sort_values('Price_USD', ascending=False).head(20)

# Display key columns for inspection
display(top20_price[['App', 'Category', 'Type', 'Price', 'Price_USD', 'Rating', 'Reviews', 'Installs', 'Genres']])

# Quick summary
print("Unique apps in top 20:", top20_price['App'].nunique())
print("Paid entries in top 20:", (top20_price['Type'] == 'Paid').sum())
print("Entries classified as GAME in top 20:", (top20_price['Category'].str.upper() == 'GAME').sum())
print("\nPrice stats for top 20:")
print(top20_price['Price_USD'].describe())


,App,Category,Type,Price,Price_USD,Rating,Reviews,Installs,Genres
2281,Vargo Anesthesia Mega App,MEDICAL,Paid,$79.99,79.99,4.60,92,1000,Medical
1407,LTC AS Legal,MEDICAL,Paid,$39.99,39.99,4.00,6,100,Medical
2629,I am Rich Person,LIFESTYLE,Paid,$37.99,37.99,4.20,134,1000,Lifestyle
2481,A Manual of Acupuncture,MEDICAL,Paid,$33.99,33.99,3.50,214,1000,Medical
4264,Golfshot Plus: Golf GPS,SPORTS,Paid,$29.99,29.99,4.10,3387,50000,Sports
504,AP Art History Flashcards,FAMILY,Paid,$29.99,29.99,5.00,1,10,Education
2463,PTA Content Master,MEDICAL,Paid,$29.99,29.99,4.20,64,1000,Medical
2207,EMT PASS,MEDICAL,Paid,$29.99,29.99,3.40,51,1000,Medical
4772,Human Anatomy Atlas 2018: Complete 3D Human Body,MEDICAL,Paid,$24.99,24.99,4.50,2921,100000,Medical
4774,Human Anatomy Atlas 2018: Complete 3D Human Body,MEDICAL,Paid,$24.99,24.99,4.50,2923,100000,Medical


Unique apps in top 20: 19
Paid entries in top 20: 20
Entries classified as GAME in top 20: 2

Price stats for top 20:
count   20.00
mean    27.91
std     14.07
min     16.99
25%     19.84
50%     24.99
75%     29.99
max     79.99
Name: Price_USD, dtype: float64


### Highest Grossing Paid Apps (ballpark estimate)

In [38]:
# 2. Create the 'Revenue_Estimate' column
# Revenue = Price_USD * Installs
df_price_clean['Revenue_Estimate'] = df_price_clean['Price_USD'] * df_price_clean['Installs']

# 3. Find the Top 10 Highest Grossing Paid Apps
top_10_grossing = df_price_clean.sort_values('Revenue_Estimate', ascending=False).head(10)

# Display the Top 10
display("--- Top 10 Highest Grossing Apps ---")
display(top_10_grossing[['App', 'Category', 'Price_USD', 'Installs', 'Revenue_Estimate']])

# 4. Count how many are Games
# Filter the top 10 for the category 'GAME'
games_count = top_10_grossing[top_10_grossing['Category'] == 'GAME'].shape[0]

print(f"\nNumber of Games in the Top 10: {games_count}")

'--- Top 10 Highest Grossing Apps ---'

,App,Category,Price_USD,Installs,Revenue_Estimate
9224,Minecraft,FAMILY,6.99,10000000,"69,900,000.00"
9220,Minecraft,FAMILY,6.99,10000000,"69,900,000.00"
8825,Hitman Sniper,GAME,0.99,10000000,"9,900,000.00"
7151,Grand Theft Auto: San Andreas,GAME,6.99,1000000,"6,990,000.00"
7477,Facetune - For Free,PHOTOGRAPHY,5.99,1000000,"5,990,000.00"
7977,Sleep as Android Unlock,LIFESTYLE,5.99,1000000,"5,990,000.00"
6594,DraStic DS Emulator,GAME,4.99,1000000,"4,990,000.00"
6082,Weather Live,WEATHER,5.99,500000,"2,995,000.00"
6856,Threema,COMMUNICATION,2.99,1000000,"2,990,000.00"
7044,Tasker,TOOLS,2.99,1000000,"2,990,000.00"



Number of Games in the Top 10: 3


# Plotly Bar Charts & Scatter Plots: Analysing App Categories

In [39]:
print(df_apps_clean.Category.nunique())
print(df_price_clean.Category.nunique())
#df_price_clean is a copy of df_apps_clean with the Price_USD and Revenue_Estimate added, just checking the categories are good

33
33


In [40]:
top10_category = df_price_clean.Category.value_counts()[:10]
display(top10_category)

Category
FAMILY             1714
GAME               1074
TOOLS               733
PRODUCTIVITY        334
FINANCE             311
PERSONALIZATION     310
COMMUNICATION       307
PHOTOGRAPHY         304
MEDICAL             302
LIFESTYLE           301
Name: count, dtype: int64

### Vertical Bar Chart - Highest Competition (Number of Apps)

In [41]:
bar = px.bar(x = top10_category.index, # index = category name
             y = top10_category.values)
 
bar.show()

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

In [43]:
category_installs = df_price_clean.groupby('Category').agg({'Installs': pd.Series.sum})
category_installs.sort_values('Installs', ascending=True, inplace=True)

In [44]:
h_bar = px.bar(x = category_installs.Installs,
               y = category_installs.index,
               orientation='h',
               title='Category Popularity')
 
h_bar.update_layout(xaxis_title='Number of Downloads', yaxis_title='Category')
h_bar.show()

### Category Concentration - Downloads vs. Competition

**Challenge**:
* First, create a DataFrame that has the number of apps in one column and the number of installs in another:

<img src=https://imgur.com/uQRSlXi.png width="350">

* Then use the [plotly express examples from the documentation](https://plotly.com/python/line-and-scatter/) alongside the [.scatter() API reference](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html)to create scatter plot that looks like this.

<img src=https://imgur.com/cHsqh6a.png>

*Hint*: Use the size, hover_name and color parameters in .scatter(). To scale the yaxis, call .update_layout() and specify that the yaxis should be on a log-scale like so: yaxis=dict(type='log')

In [46]:
# first work out the number of apps in each category
cat_number = df_price_clean.groupby('Category').agg({'App': pd.Series.count})
# use merge to combine the dataframes
cat_merged_df = pd.merge(cat_number, category_installs, on='Category', how="inner")
print(f'The dimensions of the DataFrame are: {cat_merged_df.shape}')
cat_merged_df.sort_values('Installs', ascending=False)

The dimensions of the DataFrame are: (33, 2)


,App,Installs
Category,,
GAME,1074,31543862717
COMMUNICATION,307,24152241530
SOCIAL,244,12513841475
PRODUCTIVITY,334,12463070180
TOOLS,733,11440724500
FAMILY,1714,10041105490
PHOTOGRAPHY,304,9721243130
TRAVEL_AND_LOCAL,205,6361859300
VIDEO_PLAYERS,160,6221897200


In [47]:
# create the scatter chart from above data
scatter = px.scatter(cat_merged_df, # data
                    x='App', # column name
                    y='Installs',
                    title='Category Concentration',
                    size='App',
                    hover_name=cat_merged_df.index,
                    color='Installs')
 
scatter.update_layout(xaxis_title="Number of Apps (Lower=More Concentrated)",
                      yaxis_title="Installs",
                      yaxis=dict(type='log'))
 
scatter.show()

# Extracting Nested Data from a Column

**Challenge**: How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html).


In [ ]:
# number of Genres?
num_genres = len(df_price_clean.Genres.unique())
print(f'there are {num_genres} Genres in the data.')
print(df_price_clean.Genres.value_counts().sort_values(ascending=True)[:5])

there are 115 Genres in the data.
Genres
Lifestyle;Pretend Play    1
Board;Pretend Play        1
Comics;Creativity         1
Lifestyle;Education       1
Card;Brain Games          1
Name: count, dtype: int64


In [50]:
# Split the strings on the semi-colon and then .stack them.
stack = df_price_clean.Genres.str.split(';', expand=True).stack()
print(f'We now have a single column with shape: {stack.shape}')
num_genres = stack.value_counts()
print(f'New number of genres: {len(num_genres)}')

We now have a single column with shape: (17752,)
New number of genres: 53


# Colour Scales in Plotly Charts - Competition in Genres

**Challenge**: Can you create this chart with the Series containing the genre data?

<img src=https://imgur.com/DbcoQli.png width=400>

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/).

* Find a way to set the colour scale using the color_continuous_scale parameter.
* Find a way to make the color axis disappear by using coloraxis_showscale.

In [51]:
bar = px.bar(x = num_genres.index[:15], # index = category name
             y = num_genres.values[:15], # count
             title='Top Genres',
             hover_name=num_genres.index[:15],
             color=num_genres.values[:15],
             color_continuous_scale='Agsunset')
 
bar.update_layout(xaxis_title='Genre',
yaxis_title='Number of Apps',
coloraxis_showscale=False)
 
bar.show()

# Grouped Bar Charts: Free vs. Paid Apps per Category

In [52]:
# how many free vs paid in the data
df_price_clean.Type.value_counts()

Type
Free    8278
Paid     598
Name: count, dtype: int64

In [53]:
df_free_vs_paid = df_price_clean.groupby(["Category", "Type"], as_index=False).agg({'App': pd.Series.count})
df_free_vs_paid.head()

,Category,Type,App
0,ART_AND_DESIGN,Free,59
1,ART_AND_DESIGN,Paid,3
2,AUTO_AND_VEHICLES,Free,72
3,AUTO_AND_VEHICLES,Paid,1
4,BEAUTY,Free,42


**Challenge**: Use the plotly express bar [chart examples](https://plotly.com/python/bar-charts/#bar-chart-with-sorted-or-ordered-categories) and the [.bar() API reference](https://plotly.com/python-api-reference/generated/plotly.express.bar.html#plotly.express.bar) to create this bar chart:

<img src=https://imgur.com/LE0XCxA.png>

You'll want to use the `df_free_vs_paid` DataFrame that you created above that has the total number of free and paid apps per category.

See if you can figure out how to get the look above by changing the `categoryorder` to 'total descending' as outlined in the documentation here [here](https://plotly.com/python/categorical-axes/#automatically-sorting-categories-by-name-or-total-value).

In [54]:
g_bar = px.bar(df_free_vs_paid,
               x='Category',
               y='App',
               title='Free vs Paid Apps by Category',
               color='Type',
               barmode='group')
 
g_bar.update_layout(xaxis_title='Category',
                    yaxis_title='Number of Apps',
                    xaxis={'categoryorder':'total descending'},
                    yaxis=dict(type='log'))
 
g_bar.show()

# Plotly Box Plots: Lost Downloads for Paid Apps

**Challenge**: Create a box plot that shows the number of Installs for free versus paid apps. How does the median number of installations compare? Is the difference large or small?

Use the [Box Plots Guide](https://plotly.com/python/box-plots/) and the [.box API reference](https://plotly.com/python-api-reference/generated/plotly.express.box.html) to create the following chart.

<img src=https://imgur.com/uVsECT3.png>


In [55]:
box = px.box(df_apps_clean,
             y='Installs',
             x='Type',
             color='Type',
             notched=True,
             points='all',
             title='How Many Downloads are Paid Apps Giving Up?')
 
box.update_layout(yaxis=dict(type='log'))
 
box.show()

# Plotly Box Plots: Revenue by App Category

**Challenge**: See if you can generate the chart below:

<img src=https://imgur.com/v4CiNqX.png>

Looking at the hover text, how much does the median app earn in the Tools category? If developing an Android app costs $30,000 or thereabouts, does the average photography app recoup its development costs?

Hint: I've used 'min ascending' to sort the categories.

In [56]:
df_paid_apps = df_price_clean[df_price_clean['Type'] == 'Paid']
box = px.box(df_paid_apps, 
             x='Category', 
             y='Revenue_Estimate',
             title='How Much Can Paid Apps Earn?')
 
box.update_layout(xaxis_title='Category',
                  yaxis_title='Paid App Ballpark Revenue',
                  xaxis={'categoryorder':'min ascending'},
                  yaxis=dict(type='log'))
 
 
box.show()

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.

In [61]:
print(f'The Median price for paid apps is: {df_paid_apps.Price_USD.median()}')

The Median price for paid apps is: 2.99


In [62]:
box = px.box(df_paid_apps,
             x='Category',
             y="Price",
             title='Price per Category')
 
box.update_layout(xaxis_title='Category',
                  yaxis_title='Paid App Price',
                  xaxis={'categoryorder':'max descending'},
                  yaxis=dict(type='log'))
 
box.show()